In [19]:
from napari_tomodl.processors.OPTProcessor import OPTProcessor
import numpy as np
from enum import Enum
import scipy.ndimage as ndi
import time
def min_max_normalize(image):
    return (image - image.min()) / (image.max() - image.min()) * 255


class Rec_modes(Enum):
    FBP_CPU = 0
    FBP_GPU = 1
    TWIST_CPU = 2
    UNET_GPU = 3
    MODL_GPU = 4
    MODL_CPU = 5


class Order_Modes(Enum):
    Vertical = 0
    Horizontal = 1

In [20]:
sino = np.zeros((100, 100, 100), dtype=np.float32) # angle, W, H
opt_processor = OPTProcessor()
opt_processor.resize_val = 100
opt_processor.resize_bool = False
opt_processor.register_bool = False
opt_processor.rec_process = Rec_modes.FBP_CPU.value
opt_processor.order_mode = Order_Modes.Vertical.value
opt_processor.clip_to_circle = False
opt_processor.use_filter = True
opt_processor.batch_size = 1
opt_processor.lambda_modl = 0.01

In [21]:
def reconstruct(sinos, opt_processor, resize=False, 
                input_type="3D", is_reconstruct_one=False,
                fullvolume=True, slices=0, manualalignbox=False, alignbox=0):
    """
    ToDO: Link projections
    """


    sinos = np.moveaxis(sinos, 1, 2)
    opt_processor.theta, opt_processor.Q, opt_processor.Z = sinos.shape


    if resize == True:

        optVolume = np.zeros([opt_processor.resize_val, opt_processor.resize_val, opt_processor.Z], np.float32)
        sinos = opt_processor.resize(sinos, type_sino=input_type)

    elif opt_processor.clip_to_circle == False:
        optVolume = np.zeros(
            [int(np.floor(opt_processor.Q / np.sqrt(2))), int(np.floor(opt_processor.Q / np.sqrt(2))), opt_processor.Z], np.float32
        )
    else:
        optVolume = np.zeros([opt_processor.Q, opt_processor.Q, opt_processor.Z], np.float32)

    # Reconstruction process
    # if reconstructing only one slice
    if is_reconstruct_one == True and fullvolume == False and input_type == "3D":
        slices_reconstruction = [slices]
    # if reconstructing full volume or multiple slices
    elif input_type == "3D":
        slices_reconstruction = range(opt_processor.Z if fullvolume == True else slices)
    else:
        slices_reconstruction = [0]

    batch_start = slices_reconstruction[0]
    # if use GPU process in batch to improve performance
    if opt_processor.rec_process in {Rec_modes.FBP_GPU.value, Rec_modes.MODL_GPU.value, Rec_modes.UNET_GPU.value}:
        batch_process = opt_processor.batch_size
    else:
        batch_process = 1

    batch_end = batch_start + batch_process
    # add progressBar to track the reconstruction process
    time_in = time.time()
    while batch_start <= slices_reconstruction[-1]:
        print("Reconstructing slices {} to {}".format(batch_start, batch_end), end="\r")
        zidx = slice(batch_start, batch_end)
        ####################### stacks reconstruction ############################
        if input_type == "3D":
            if opt_processor.register_bool == True:

                # sinos[:, :, zidx] = cv2.normalize(
                #     sinos[:, :, zidx], None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F
                # )
                sinos[:, :, zidx] = min_max_normalize(sinos[:, :, zidx])
                if opt_processor.order_mode == 0:
                    optVolume[:, :, zidx] = opt_processor.correct_and_reconstruct(sinos[:, :, zidx].transpose(1, 0, 2))
                elif opt_processor.order_mode == 1:
                    optVolume[:, :, zidx] = opt_processor.correct_and_reconstruct(sinos[:, :, zidx])
                sinos[:, :, zidx] = min_max_normalize(sinos[:, :, zidx])

            elif manualalignbox == True:
                sinos[:, :, zidx] = min_max_normalize(sinos[:, :, zidx])
                if opt_processor.order_mode == 0:
                    optVolume[:, :, zidx] = opt_processor.reconstruct(
                        ndi.shift(sinos[:, :, zidx], (0, alignbox, 0), mode="nearest").transpose(
                            1, 0, 2
                        )
                    )
                elif opt_processor.order_mode == 1:
                    optVolume[:, :, zidx] = opt_processor.reconstruct(
                        ndi.shift(sinos[:, :, zidx], (alignbox, 0, 0), mode="nearest")
                    )
                sinos[:, :, zidx] = min_max_normalize(sinos[:, :, zidx])

            else:
                sinos[:, :, zidx] = min_max_normalize(sinos[:, :, zidx])
                if opt_processor.order_mode == 0:
                    optVolume[:, :, zidx] = opt_processor.reconstruct(sinos[:, :, zidx].transpose(1, 0, 2))

                elif  opt_processor.order_mode == 1:
                    optVolume[:, :, zidx] = opt_processor.reconstruct(sinos[:, :, zidx])
                sinos[:, :, zidx] = min_max_normalize(sinos[:, :, zidx])

        ####################### 2D reconstruction ############################
        elif input_type == "2D":
            if opt_processor.register_bool == True:
                optVolume[:, :, zidx] = opt_processor.correct_and_reconstruct(sinos[:, :, zidx])

            elif manualalignbox == True:
                optVolume[:, :, zidx] = opt_processor.reconstruct(
                    ndi.shift(sinos[:, :, zidx], (alignbox, 0, 0), mode="nearest")
                )
            else:
                optVolume[:, :, zidx] = opt_processor.reconstruct(sinos[:, :, zidx])

        batch_start = batch_end
        batch_end += batch_process
    print("Computation time total: {} s".format(round(time.time() - time_in, 3)))

    if is_reconstruct_one == True and fullvolume == False and input_type == "3D":
        return optVolume[..., slices]
    elif input_type == "3D":
        return np.rollaxis(optVolume, -1)
    else:
        return optVolume[..., 0]

In [22]:
# shape of the sinogram  angle, W, H
sinogram = np.zeros((400, 100, 200))


In [23]:
opt_volume = reconstruct(sinogram, opt_processor)

C:\Users\Predator\AppData\Local\Temp\ipykernel_18876\4220327519.py:7: RuntimeWarning: invalid value encountered in divide
  return (image - image.min()) / (image.max() - image.min()) * 255


KeyboardInterrupt: 